<a href="https://colab.research.google.com/github/pankaj18/NLP-Projects/blob/master/twitter_sentiment_nltk/twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://monkeylearn.com/sentiment_analysis
https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk

In [1]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('dark_background')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


download sample tweets from nltk package

In [2]:
nltk.download('twitter_samples')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from nltk.corpus import twitter_samples
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
import re
import string

In [4]:
positive_tweets=twitter_samples.strings('positive_tweets.json') #5000 tweets with pos sentiment
negative_tweets=twitter_samples.strings('negative_tweets.json')#5000 tweets with neg sentiment
text=twitter_samples.strings('tweets.20150430-223406.json')#20000 tweets with no sentiment

Strings() method of twitter_samples will print all of the tweets within dataest as strings

punkt module is pretrained model that helps for tokenize words

In [5]:
tweet_tokens=twitter_samples.tokenized('positive_tweets.json')


In [6]:
print(tweet_tokens[0])

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':)']


In [7]:
print(pos_tag(tweet_tokens[0]))

[('#FollowFriday', 'JJ'), ('@France_Inte', 'NNP'), ('@PKuchly57', 'NNP'), ('@Milipol_Paris', 'NNP'), ('for', 'IN'), ('being', 'VBG'), ('top', 'JJ'), ('engaged', 'VBN'), ('members', 'NNS'), ('in', 'IN'), ('my', 'PRP$'), ('community', 'NN'), ('this', 'DT'), ('week', 'NN'), (':)', 'NN')]


#Normalizing Data

bring data to base form. the function lemantize_srntence first get position token of a tweet. 

In [8]:
def lemmatize_sentence(tokens):
  lemmatizer=WordNetLemmatizer()
  lemmatized_sentence=[]
  for word,tag in pos_tag(tokens):
    print(word,tag)
    #print(tag)
    if tag.startswith('NN'):
      pos='n'
    elif tag.startswith('VB'):
      pos='v'
    else:
      pos='a'
    lemmatized_sentence.append(lemmatizer.lemmatize(word,pos))
  return lemmatized_sentence

In [9]:
print(lemmatize_sentence(tweet_tokens[0]))

#FollowFriday JJ
@France_Inte NNP
@PKuchly57 NNP
@Milipol_Paris NNP
for IN
being VBG
top JJ
engaged VBN
members NNS
in IN
my PRP$
community NN
this DT
week NN
:) NN
['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']


word being changes to be and members changes to member

In [10]:
lemmatizer=WordNetLemmatizer()
lemmatizer.lemmatize('being','v')

'be'

Removing noise from data
* Hyperlinks
* Twitter handles
* Punctuations

In [11]:
string=" Hi i am pankaj. my email id is pankaj.eln@gmail.com. @hawa. i wnt to check website http://google.com and https://google.co.in"

In [12]:
pattern1="http:\S+"
pattern2="https:\S+"
pattern3="@[a-zA-Z0-9]+"

In [13]:
from nltk.tokenize import word_tokenize

In [14]:
word_tokenize(string)

['Hi',
 'i',
 'am',
 'pankaj',
 '.',
 'my',
 'email',
 'id',
 'is',
 'pankaj.eln',
 '@',
 'gmail.com',
 '.',
 '@',
 'hawa',
 '.',
 'i',
 'wnt',
 'to',
 'check',
 'website',
 'http',
 ':',
 '//google.com',
 'and',
 'https',
 ':',
 '//google.co.in']

In [15]:
s1=re.sub(pattern1,"",string)
s1

' Hi i am pankaj. my email id is pankaj.eln@gmail.com. @hawa. i wnt to check website  and https://google.co.in'

In [16]:
s2=re.sub(pattern2," ",s1)
s2

' Hi i am pankaj. my email id is pankaj.eln@gmail.com. @hawa. i wnt to check website  and  '

In [17]:
s3=re.sub(pattern3," ",s2)
s3

' Hi i am pankaj. my email id is pankaj.eln .com.  . i wnt to check website  and  '

In [18]:
import string

In [19]:
def remove_noise(tweet_tokens,stop_words=()):
  cleaned_tokens=[]
  for token,tag in pos_tag(tweet_tokens):
    token=re.sub('http:\S+',"",token)
    token=re.sub('https:\S+',"",token)
    token=re.sub('@[a-zA-Z0-9_]+',"",token)
    if tag.startswith("NN"):
      pos="n"
    elif tag.startswith("VB"):
      pos="v"
    else:
      pos="a"
    lemmatizer=WordNetLemmatizer()
    token=lemmatizer.lemmatize(token,pos)
    if ((len(token)>0) and (token not in string.punctuation) and (token.lower() not in stop_words)):
      cleaned_tokens.append(token.lower())
  return cleaned_tokens  


In [20]:
stop_words=stopwords.words('english')

In [21]:
for i in range(10):
  print(remove_noise(tweet_tokens[i],stop_words))

['#followfriday', 'top', 'engage', 'member', 'community', 'week', ':)']
['hey', 'james', 'odd', ':/', 'please', 'call', 'contact', 'centre', '02392441234', 'able', 'assist', ':)', 'many', 'thanks']
['listen', 'last', 'night', ':)', 'bleed', 'amazing', 'track', 'scotland']
['congrats', ':)']
['yeaaaah', 'yippppy', 'accnt', 'verify', 'rqst', 'succeed', 'get', 'blue', 'tick', 'mark', 'fb', 'profile', ':)', '15', 'day']
['one', 'irresistible', ':)', '#flipkartfashionfriday']
['like', 'keep', 'lovely', 'customer', 'wait', 'long', 'hope', 'enjoy', 'happy', 'friday', 'lwwf', ':)']
['second', 'thought', '’', 'enough', 'time', 'dd', ':)', 'new', 'short', 'enter', 'system', 'sheep', 'must', 'buy']
['jgh', 'go', 'bayan', ':d', 'bye']
['act', 'mischievousness', 'call', 'etl', 'layer', 'in-house', 'warehouse', 'app', 'katamari', 'well', '…', 'name', 'imply', ':p']


In [22]:
tweet_tokens[0]

['#FollowFriday',
 '@France_Inte',
 '@PKuchly57',
 '@Milipol_Paris',
 'for',
 'being',
 'top',
 'engaged',
 'members',
 'in',
 'my',
 'community',
 'this',
 'week',
 ':)']

use remove_noise function to clean positive and negative tweets

In [23]:
pos_tweet_tokens=twitter_samples.tokenized('positive_tweets.json')
neg_tweet_tokens=twitter_samples.tokenized('negative_tweets.json')
positive_cleaned_token_list=[]
negative_cleaned_token_list=[]
for tokens in pos_tweet_tokens:
  positive_cleaned_token_list.append(remove_noise(tokens,stop_words))
for tokena in neg_tweet_tokens:
  negative_cleaned_token_list.append(remove_noise(tokens,stop_words))




In [24]:
print(pos_tweet_tokens[500])
print(positive_cleaned_token_list[500])

['Dang', 'that', 'is', 'some', 'rad', '@AbzuGame', '#fanart', '!', ':D', 'https://t.co/bI8k8tb9ht']
['dang', 'rad', '#fanart', ':d']


# determine word density

analyze word frequency of positive tweets

In [25]:
def get_all_words(cleaned_tokens_list):
  for tokens in cleaned_tokens_list:
    for token in tokens:
      yield token

In [26]:
all_pos_words=get_all_words(positive_cleaned_token_list)

In [27]:
next(all_pos_words)

'#followfriday'

In [28]:
freq_dist_pos=FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


In [29]:
positive_cleaned_token_list[0]

['#followfriday', 'top', 'engage', 'member', 'community', 'week', ':)']

# Preparing Data for Model

## Converting tokens into dictionary
To implement Naive bayes classifier model. this requires python dictionary with word as keys and True as values.

In [30]:
def get_tweets_for_model(cleaned_tokens_list):
  for tweet_token in cleaned_tokens_list:
    yield dict([token, True] for token in tweet_token)

In [31]:
positive_tokens_for_model=get_tweets_for_model(positive_cleaned_token_list)
negative_tokens_for_model=get_tweets_for_model(negative_cleaned_token_list)

In [32]:
next(positive_tokens_for_model)

{'#followfriday': True,
 ':)': True,
 'community': True,
 'engage': True,
 'member': True,
 'top': True,
 'week': True}

Splitting Dataset for training and testing Model

In [33]:
import random
positive_dataset=[(tweet_dict,"Positive") for tweet_dict in positive_tokens_for_model]
negative_dataset=[(tweet_dict,"Negative") for tweet_dict in negative_tokens_for_model]
dataset=positive_dataset+negative_dataset
random.shuffle(dataset)
train_data=dataset[:7000]
test_data=dataset[7000:]

In [34]:
# this code attaches positive and negative labels to each tweet.
# it creates dataset by joining positive and negative tweets.
# divide dataset into 70:30 ratio
train_data[0]

({':-)': True, 'could': True, 'egg': True, 'face': True, 'say': True},
 'Negative')

# Building and Testing Model

In [35]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier=NaiveBayesClassifier.train(train_data)
print("accuracy is:",classify.accuracy(classifier,test_data))
print(classifier.show_most_informative_features(20))

accuracy is: 0.995998666222074
Most Informative Features
                     egg = True           Negati : Positi =   1402.6 : 1.0
                    face = True           Negati : Positi =    369.1 : 1.0
                   could = True           Negati : Positi =    149.2 : 1.0
                     say = True           Negati : Positi =     64.3 : 1.0
                     :-) = True           Negati : Positi =      7.6 : 1.0
                      :) = None           Negati : Positi =      3.5 : 1.0
                      :d = None           Negati : Positi =      1.1 : 1.0
                  thanks = None           Negati : Positi =      1.1 : 1.0
                  follow = None           Negati : Positi =      1.1 : 1.0
                    love = None           Negati : Positi =      1.1 : 1.0
                     ... = None           Negati : Positi =      1.1 : 1.0
                    good = None           Negati : Positi =      1.1 : 1.0
                     get = None           N

# Testing Classifier

In [38]:
from nltk.tokenize import word_tokenize
custom_tweet="I ordered just one from flipkart,they screwed up, never used the app again"
custom_tokens=remove_noise(word_tokenize(custom_tweet))
print(classifier.classify(dict([token,True] for token in custom_tokens)))

Positive
